In [10]:
import math
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils import data




In [3]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
class ResNet(nn.Module):

    def __init__(self, depth, num_classes=1000, block_name='BasicBlock'):
        super(ResNet, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        if block_name.lower() == 'basicblock':
            assert (depth - 2) % 6 == 0, 'When use basicblock, depth should be 6n+2, e.g. 20, 32, 44, 56, 110, 1202'
            n = (depth - 2) // 6
            block = BasicBlock
        elif block_name.lower() == 'bottleneck':
            assert (depth - 2) % 9 == 0, 'When use bottleneck, depth should be 9n+2, e.g. 20, 29, 47, 56, 110, 1199'
            n = (depth - 2) // 9
            block = Bottleneck
        else:
            raise ValueError('block_name shoule be Basicblock or Bottleneck')


        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, n)
        self.layer2 = self._make_layer(block, 32, n, stride=2)
        self.layer3 = self._make_layer(block, 64, n, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)    # 32x32

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16
        x = self.layer3(x)  # 8x8

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet32(**kwargs):
    model = ResNet(depth=32, **kwargs)
    return model

In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# @neelabh17 implementation


class CCELossFast(torch.nn.Module):
    def __init__(self, n_classes, n_bins = 10, mode = "eval", loss_type = "sce"):
        '''
        deprecated info ignore!!
        output = [n_Class, h , w] np array: The complete probability vector of an image
        target = [h , w] np array: The GT for the image
        n_bins = [h , w] np array: Number of bins for the Calibration division

        '''
        super(CCELossFast,self).__init__()
        self.n_classes = n_classes
        self.n_bins = n_bins
        self.mode = mode
        self.loss_type = loss_type


        self.createBins()

        self.no_pred_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        self.no_acc_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        self.conf_sum_tot = torch.zeros(self.n_classes, self.n_bins).cuda()

    def reset(self):
        self.no_pred_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        self.no_acc_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        self.conf_sum_tot = torch.zeros(self.n_classes, self.n_bins).cuda()


    def forward(self , output, target):
        '''
        imp info!! dont ignore
        output = [batch, n_Class] np array: The complete logit vector of an image 

        target = [batch] np array: The GT for the image

        create an three array of [n_class, n_bins]
        -> Number of prediciton array for that specification
        -> Number of correct prediction for that class
        -> Percentge of correct 
        '''

        current_no_pred_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        current_no_acc_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        current_conf_sum_tot = torch.zeros(self.n_classes, self.n_bins).cuda()
        

        output = torch.softmax(output, dim=1)
        # [batch, classes]
        
        for i, (bin_lower, bin_upper) in enumerate(zip(self.bin_lowers, self.bin_uppers)):
            mask = (output> bin_lower) * (output <= bin_upper)

            for class_id in range(self.n_classes):


                class_mask = mask[:,class_id]

                classwise_gt = (target == class_id).long()

                current_no_pred_tot[class_id][i] = torch.sum(class_mask)
                current_no_acc_tot[class_id][i] = torch.sum(class_mask *  classwise_gt)
                current_conf_sum_tot[class_id][i] = torch.sum((output[:,class_id])[class_mask])


        self.no_pred_tot += current_no_pred_tot.data
        self.no_acc_tot += current_no_acc_tot.data
        self.conf_sum_tot += current_conf_sum_tot.data

        avg_acc = (current_no_acc_tot)/(current_no_pred_tot + 1e-13)
        avg_conf = current_conf_sum_tot / (current_no_pred_tot + 1e-13)
        # overall_cceLoss = torch.sum(torch.abs(avg_acc - avg_conf) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))
        # overall_cceLoss = torch.sum(((avg_acc - avg_conf)**2))

        assert (self.loss_type=="sce" or self.loss_type=="kernel" or self.loss_type=="diff")

        # Correct implementation
        if(self.loss_type=="sce"):
            overall_cceLoss = torch.sum(torch.abs(avg_acc - avg_conf) * current_no_pred_tot/torch.sum(current_no_pred_tot))

        # Kernel based implementation
        elif(self.loss_type=="kernel"):
            overall_cceLoss = torch.sum((1-torch.exp((-1*((avg_acc - avg_conf)**2))/0.5)) * (current_no_pred_tot/torch.sum(current_no_pred_tot)))
            
        # difference based approach
        elif(self.loss_type=="diff"):
            overall_cceLoss = torch.sum(((avg_acc - avg_conf)**2))

        # overall_cceLoss = torch.sum(12500*(1-torch.exp((-1*((avg_acc - avg_conf)**2))/6400)) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))
        # print(self.conf_sum_tot.requires_grad)
        return overall_cceLoss

    def createBins(self):

        #uniform bin spacing
        
        bin_boundaries = np.linspace(0, 1, self.n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]
        self.avg_bin = torch.Tensor((self.bin_lowers + self.bin_uppers)/2).cuda()
        

    def get_perc_table(self, classes):
        self.perc = (self.no_acc_tot)/(self.no_pred_tot + 1e-13)
        self.perc *= 100
        
        from tabulate import tabulate
        x= list(self.perc.cpu().numpy())

        for i in range(len(x)):
            x[i]=list(x[i])
            x[i]=[classes[i]]+list(x[i])
        print(tabulate(x, headers = ["Classes"]+[ "{:0.2f} - {:0.2f}".format(self.bin_lowers[i] * 100, self.bin_uppers[i] * 100) for i in range( len(self.bin_lowers))]))
        
        return self.perc

    def get_diff_score(self):
        avg_acc = (self.no_acc_tot)/(self.no_pred_tot + 1e-13)
        avg_conf = self.conf_sum_tot / (self.no_pred_tot + 1e-13)
        return torch.sum (torch.abs(avg_acc-avg_conf))/(self.n_bins*self.n_classes)

    def get_overall_CCELoss(self):
        avg_acc = (self.no_acc_tot)/(self.no_pred_tot + 1e-13)
        avg_conf = self.conf_sum_tot / (self.no_pred_tot + 1e-13)
        # overall_cceLoss = torch.sum(torch.abs(avg_acc - avg_conf) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))
        # overall_cceLoss = torch.sum(((avg_acc - avg_conf)**2))

        # Correct implementation
        # overall_cceLoss = torch.sum(((avg_acc - avg_conf)**2) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))

        # Non Squared  implementation
        overall_cceLoss = torch.sum((torch.abs(avg_acc - avg_conf)) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))

        # Kernel based implementation
        # overall_cceLoss = torch.sum((1-torch.exp((-1*((avg_acc - avg_conf)**2))/0.5)) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))
        # overall_cceLoss = torch.sum(12500*(1-torch.exp((-1*((avg_acc - avg_conf)**2))/6400)) * (self.no_pred_tot/torch.sum(self.no_pred_tot)))

        # print("Overall CCE Loss = ", overall_cceLoss)

        return overall_cceLoss

        
    def get_classVise_CCELoss(self, classes):
        avg_acc = (self.no_acc_tot)/(self.no_pred_tot + 1e-13)
        # print(avg_acc.shape)
        avg_conf = self.conf_sum_tot / (self.no_pred_tot + 1e-13)
        # print(avg_conf.shape)

        x = torch.sum(torch.abs(avg_acc-avg_conf) * self.no_pred_tot, dim = 1) / torch.sum(self.no_pred_tot, dim = 1)
        x = x.reshape(-1,1)

        # print(x.shape)

        x=list(x)
        from tabulate import tabulate
        for i in range(len(x)):
            x[i]=list(x[i])
            x[i]=[classes[i]]+list(x[i])
        print(tabulate(x, headers = ["Classes", "ECELoss"]))

    def get_diff_mean_std (self):
        self.perc = (self.no_acc_tot)/(self.no_pred_tot + 1e-13)
        avg_conf = self.conf_sum_tot / (self.no_pred_tot + 1e-13)
        self.perc *= 100
        avg_conf *= 100
        dif = torch.abs(avg_conf- self.perc)
        return dif.mean(), dif.std()


In [7]:
num_classes = 10
model = resnet32(num_classes= num_classes)
resume = "checkpoint.pth"
saved_model_dict = torch.load(resume)
model.load_state_dict(saved_model_dict['state_dict'])
model.cuda()
sce_criterion = CCELossFast(n_classes = num_classes)



In [8]:
#preparing dataset
transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
dataloader = datasets.CIFAR10

testset = dataloader(root='./data', train=False, download=False, transform=transform_test)
testloader = data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=8)



In [19]:
model.eval()
bar = tqdm(testloader, total=len(testloader))

sce_criterion.reset()
with torch.no_grad():
    for inputs, targets in bar:
        targets = targets.cuda()
        inputs = inputs.cuda()
        # compute output
        outputs = model(inputs)
        sce_criterion.forward(outputs, targets)


sce = sce_criterion.get_overall_CCELoss().item()
print()
print(sce)

        

100%|██████████| 79/79 [00:03<00:00, 22.21it/s]
0.035657722502946854

